<a href="https://colab.research.google.com/github/SvgPrizrak/ML_Sber_Homework/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22hw03_regression_models%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашняя работа 3. Валидация алгоритмов регрессии

In [ ]:
# поскольку на новой версии scikit-learn load_boston отсутствует, то проще поставить более раннюю версию пакета
!pip install scikit-learn==1.1.3

In [ ]:
# Импорт базовых библиотек
import numpy as np
import pandas as pd

# игнорирование исключений (будущих или вообще всех)
import warnings
# warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

# Визуализация
import seaborn as sns
import matplotlib.pyplot as plt

# Датасеты
from sklearn.datasets import load_boston  # для регрессии

# Utils
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV

# Модели (регрессия)
from sklearn.linear_model import SGDRegressor, LinearRegression, Lasso, Ridge
from sklearn.svm import SVR

# Метрики
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score  # для регрессии

# во избежание эффекта масштаба при валидации модели импортируем StandardScaler
from sklearn.preprocessing import StandardScaler

%matplotlib inline

# Регрессия

In [ ]:
# Загрузим данные
data = load_boston()
dir(data)

['DESCR', 'data', 'data_module', 'feature_names', 'filename', 'target']

In [ ]:
type(data)

sklearn.utils._bunch.Bunch

In [ ]:
# Явно "достанем" из данных объекты и метки

X = data.data
y = data.target

# "скалирование" X (часть из моделей адекватно работает только в одном масштабе)
sc = StandardScaler()
X_scaled = sc.fit_transform(X)

print(f'Type X: {type(X)}, X shape: {X.shape}')
print(f'Type y: {type(y)}, y shape: {y.shape}')
print(data.feature_names)

Type X: <class 'numpy.ndarray'>, X shape: (506, 13)
Type y: <class 'numpy.ndarray'>, y shape: (506,)
['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']


### Задача 1. Подготовка данных

Сконвертируйте набора данных в `pandas.Dataframe`:

In [ ]:
# Ваш код
boston_df = pd.DataFrame(np.hstack([X_scaled, y.reshape(-1, 1)]), columns = np.append(data.feature_names, ['target']))
boston_df.columns = map(str.lower, boston_df.columns)
boston_df

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,target
0,-0.419782,0.284830,-1.287909,-0.272599,-0.144217,0.413672,-0.120013,0.140214,-0.982843,-0.666608,-1.459000,0.441052,-1.075562,24.0
1,-0.417339,-0.487722,-0.593381,-0.272599,-0.740262,0.194274,0.367166,0.557160,-0.867883,-0.987329,-0.303094,0.441052,-0.492439,21.6
2,-0.417342,-0.487722,-0.593381,-0.272599,-0.740262,1.282714,-0.265812,0.557160,-0.867883,-0.987329,-0.303094,0.396427,-1.208727,34.7
3,-0.416750,-0.487722,-1.306878,-0.272599,-0.835284,1.016303,-0.809889,1.077737,-0.752922,-1.106115,0.113032,0.416163,-1.361517,33.4
4,-0.412482,-0.487722,-1.306878,-0.272599,-0.835284,1.228577,-0.511180,1.077737,-0.752922,-1.106115,0.113032,0.441052,-1.026501,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,-0.413229,-0.487722,0.115738,-0.272599,0.158124,0.439316,0.018673,-0.625796,-0.982843,-0.803212,1.176466,0.387217,-0.418147,22.4
502,-0.415249,-0.487722,0.115738,-0.272599,0.158124,-0.234548,0.288933,-0.716639,-0.982843,-0.803212,1.176466,0.441052,-0.500850,20.6
503,-0.413447,-0.487722,0.115738,-0.272599,0.158124,0.984960,0.797449,-0.773684,-0.982843,-0.803212,1.176466,0.441052,-0.983048,23.9
504,-0.407764,-0.487722,0.115738,-0.272599,0.158124,0.725672,0.736996,-0.668437,-0.982843,-0.803212,1.176466,0.403225,-0.865302,22.0


### Задача 2. Обучение моделей с помощью кросс-валидации

Обучите алгоритмы следующие регрессии на наборе данных:
- `sklearn.linear_model.SGDRegressor`
- `sklearn.linear_model.Lasso`
- `sklearn.svm.SVR`

Используйте k-fold кросс-валидацию (`cross_val_score`). Сравните результаты. Для оценки качества алгоритмов используйте `mean_absolute_error`, `mean_squared_error`.

In [ ]:
# Ваш код
# инициализации экземпляров классов
sgdreg, lr, lasso, ridge, svr = SGDRegressor(), LinearRegression(), Lasso(), Ridge(), SVR()

# напишем функцию для выявления качества алгоритмов (KFold CV)
def k_fold_cross_validation(X: np.array, y: np.array, estimators_list: list, cvs_skf_list: list, scorings_list: list):

    '''
    Функция создает датафрейм с метриками (MAE, MSE и R2_score) для различных алгоритмов машинного обучения при KFold кросс-валидации:

    Args:
    * X - массив входных признаков;
    * y - массив целевых признаков;
    * estimators_list - список алгоритмов машинного обучения (экзмеляров класса);
    * cvs_skf_list - список количества разделений X и y для KFold CV;
    * scorings_list - список метрик машинного обучения для валидации;

    Returns:
    * датафрейм со всеми метриками из вышеперечисленных списков
    '''

    # создадим датафрейм и пройдемся циклом для добавления метрик в него
    metrics_quality_df = pd.DataFrame()
    for estimator in estimators_list:
        for skf in cvs_skf_list:
            for scoring in scorings_list:
                part_df = pd.DataFrame({'estimator': [estimator], 'cv_skf_value': [skf], 'scoring': scoring,
                                        'mean_scoring_value': np.mean(cross_val_score(estimator, X=X, y=y, cv=KFold(n_splits=skf, shuffle=True, random_state=42), scoring=scoring))})
                metrics_quality_df = pd.concat([metrics_quality_df, part_df])

    # манипуляции над столбцом estimator
    metrics_quality_df['estimator'] = metrics_quality_df['estimator'].astype('str')
    metrics_quality_df['estimator'] = metrics_quality_df['estimator'].apply(lambda x: x[:-2])

    # создание сводной таблицы для анализа данных
    metrics_quality_df = pd.pivot_table(data = metrics_quality_df, index=['cv_skf_value', 'scoring', 'estimator']).unstack(['estimator'])

    return metrics_quality_df

In [ ]:
# проверка работоспособности функции
k_fold_cross_validation_df = k_fold_cross_validation(X_scaled, y, estimators_list=[sgdreg, lr, lasso, ridge, svr],
                                                     cvs_skf_list=[3, 5, 7, 9], scorings_list = ['r2', 'neg_mean_absolute_error', 'neg_mean_squared_error'])
k_fold_cross_validation_df

mean_scoring_value                   \
estimator                                         Lasso LinearRegression   
cv_skf_value scoring                                                       
3            neg_mean_absolute_error          -3.732193        -3.380921   
             neg_mean_squared_error          -29.046227       -23.193755   
             r2                                0.654692         0.724296   
5            neg_mean_absolute_error          -3.745835        -3.388460   
             neg_mean_squared_error          -29.315633       -23.488596   
             r2                                0.647449         0.715222   
7            neg_mean_absolute_error          -3.753702        -3.414723   
             neg_mean_squared_error          -29.297215       -23.907128   
             r2                                0.650629         0.711698   
9            neg_mean_absolute_error          -3.761186        -3.430270   
             neg_mean_squared_error          -29.665723       -24.242062   
             r2                                0.639031         0.699494   

                                                                         
estimator                                 Ridge SGDRegressor        SVR  
cv_skf_value scoring                                                     
3            neg_mean_absolute_error  -3.375217    -3.377242  -3.298493  
             neg_mean_squared_error  -23.190196   -23.457711 -31.801028  
             r2                        0.724355     0.724157   0.623535  
5            neg_mean_absolute_error  -3.383271    -3.380522  -3.179419  
             neg_mean_squared_error  -23.488136   -23.694567 -29.870827  
             r2                        0.715233     0.713468   0.643380  
7            neg_mean_absolute_error  -3.409536    -3.397945  -3.133991  
             neg_mean_squared_error  -23.896489   -24.180803 -28.925795  
             r2                        0.711868     0.712589   0.659538  
9            neg_mean_absolute_error  -3.424903    -3.423130  -3.121187  
             neg_mean_squared_error  -24.229864   -24.322966 -28.863040  
             r2                        0.699646     0.699248   0.655404

Исходя из таблицы среди 5 алгоритмов лучше всего выбрать алгоритм `Ridge` с более устойчивой ошибкой MSE, MAE и R2_score. Если же рассматривать только 3 алгоритма из списка, то лучше всего использовать `SGDRegressor` - наилучшее соотношение MSE, MAE и R2_score (однако, если выбирать только по MAE, то следует выбрать `SVR`).

### Задача 3. Дополнительная валидация

Повторите тоже самое только для Leave-one-out валидации:

In [ ]:
# Ваш код
from sklearn.model_selection import LeaveOneOut

# инициализация класса
loo = LeaveOneOut()

# напишем функцию для выявления качества алгоритмов (Leave_One_Out CV)
def leave_one_out_cross_validation(X: np.array, y: np.array, estimators_list: list, scorings_list: list):

    '''
    Функция создает датафрейм с метриками (MAE, MSE) для различных алгоритмов машинного обучения при Leave_One_Out кросс-валидации:

    Args:
    * X - массив входных признаков;
    * y - массив целевых признаков;
    * estimators_list - список алгоритмов машинного обучения (экзмепляров класса);
    * scorings_list - список метрик машинного обучения для валидации;

    Returns:
    * датафрейм со всеми метриками из вышеперечисленных списков
    '''

    # создадим датафрейм и пройдемся циклом для добавления метрик в него
    metrics_quality_df = pd.DataFrame()

    for estimator in estimators_list:
        for scoring in scorings_list:
            part_df = pd.DataFrame({'estimator': [estimator], 'scoring': scoring,
                                    'mean_scoring_value': np.mean(cross_val_score(estimator, X=X, y=y, cv=loo, scoring=scoring))})
            metrics_quality_df = pd.concat([metrics_quality_df, part_df])

    # манипуляции над столбцом estimator
    metrics_quality_df['estimator'] = metrics_quality_df['estimator'].astype('str')
    metrics_quality_df['estimator'] = metrics_quality_df['estimator'].apply(lambda x: x[:-2])

    # создание сводной таблицы для анализа данных
    metrics_quality_df = pd.pivot_table(data = metrics_quality_df, index=['scoring', 'estimator']).unstack(['estimator'])

    return metrics_quality_df

In [ ]:
# проверка работоспособности функции
loo_cross_validation_df = leave_one_out_cross_validation(X_scaled, y, estimators_list=[sgdreg, lr, lasso, ridge, svr],
                                                         scorings_list = ['neg_mean_absolute_error', 'neg_mean_squared_error'])

loo_cross_validation_df

mean_scoring_value                              \
estimator                            Lasso LinearRegression      Ridge   
scoring                                                                  
neg_mean_absolute_error          -3.758884        -3.382797  -3.378657   
neg_mean_squared_error          -29.442926       -23.725746 -23.718113   

                                                 
estimator               SGDRegressor        SVR  
scoring                                          
neg_mean_absolute_error    -3.376709  -3.020454  
neg_mean_squared_error    -24.036832 -27.321963

>**Info - Leave-one-out валидация**
>
>Leave-one-out валидация - это кросс-валидация с кол-вом фолдов, равным количеству объектов ($K=N$). По сути вы обучаете N раз, каждый раз на N-1 объектах, а затем валидируетесь на оставшемся 1 объекте.
>
>Название класса, который необходим для выполнения этого задания, нужно найти вам самим :)

Исходя из таблицы среди 5 алгоритмов лучше всего выбрать алгоритм `SGDRegressor` или `Ridge` с более устойчивой ошибкой MSE, MAE (однако, если выбирать только по MAE, то следует выбрать `SVR`). Значения метрик в целом близки к значениям, полученным через KFold.

### Задача 4. Подбор гиперпараметров

Выберите по два гиперпараметра у каждой модели из приведённых выше. Настройте гиперпараметры алгоритмов при помощи `GridSearchCV`:

>**Примечание**
>
>Выбирайте имеющие смыслы гиперпараметры, которые могут внести ощутимый вклад. Например, не стоит перебирать `random_state`, в этом нет прикладного смысла.

In [ ]:
# Ваш код

# напишем функцию для создания датафрейма с настроенными гиперпараметрами
def make_grid_search_cv(X: np.array, y: np.array, estimator, train_size: float, random_state: int, param_grid: dict, cvs_list: list, scorings_list: list, verbose: int):

    '''
    Функция создает датафрейм с метриками (MAE, MSE, R2) для различных алгоритмов машинного обучения при использовании GridSearchCV:

    Args:
    * X - массив входных признаков;
    * y - массив целевых признаков;
    * estimator - алгоритм машинного обучения;
    * train_size - размер тестовой выборки в долях;
    * random_state - размер тестовой выборки в долях;
    * param_grid - словарь с гиперпараметрами алгоритма машинного обучения;
    * cvs_list - список количества разделений X и y для GridSearchCV;
    * scorings_list - список метрик машинного обучения для валидации;
    * verbose - параметр отображения процесса обучения;


    Returns:
    * датафрейм со всеми метриками из вышеперечисленных списков
    '''

    # для валидации разделим датафрейм на обучающую и тестовую части
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size, random_state=random_state)
    X_train_scaled = sc.fit_transform(X_train)
    X_test_scaled = sc.transform(X_test)

    # создадим датафрейм и пройдемся циклом для добавления метрик в него
    metrics_quality_df = pd.DataFrame()

    for cv in cvs_list:
        for scoring in scorings_list:

            # инициализация класса
            print("-" * 110)
            clf = GridSearchCV(estimator=estimator, param_grid=param_grid, cv=cv, scoring=scoring, verbose=verbose, n_jobs=-1)

            # обучение модели
            clf.fit(X_train_scaled, y_train)

            # получение лучшего варианта estimator
            best_estimator = clf.best_estimator_
            print(f'CV: {cv}, метрика: {scoring}')
            print(f'Лучшие параметры модели при переборе на обучающей выборке: {clf.best_params_}')

            # обучение на лучшем варианте модели и рассмотрение метрик на обучающей и тестовой части в зависимости от scoring
            best_estimator.fit(X_train_scaled, y_train)

            y_train_pred = best_estimator.predict(X_train_scaled)
            y_test_pred = best_estimator.predict(X_test_scaled)

            if scoring == 'neg_mean_absolute_error':
                train_metric, test_metric = mean_absolute_error(y_train, y_train_pred), mean_absolute_error(y_test, y_test_pred)
                print(f'MAE на обучающей части: {train_metric}')
                print(f'MAE на тестовой части: {test_metric}')

            elif scoring == 'neg_mean_squared_error':
                train_metric, test_metric = mean_squared_error(y_train, y_train_pred), mean_squared_error(y_test, y_test_pred)
                print(f'MSE на обучающей части: {train_metric}')
                print(f'MSE на тестовой части: {test_metric}')

            else:
                train_metric, test_metric = r2_score(y_train, y_train_pred), r2_score(y_test, y_test_pred)
                print(f'R2 score: {train_metric}')
                print(f'R2 score: {test_metric}')

            part_df = pd.DataFrame({'estimator': [estimator], 'scoring': scoring, 'cv': [cv], 'train_metric_value': train_metric, 'test_metric_value': test_metric})
            metrics_quality_df = pd.concat([metrics_quality_df, part_df])

    # манипуляции над столбцом estimator
    metrics_quality_df['estimator'] = metrics_quality_df['estimator'].astype('str')
    metrics_quality_df['estimator'] = metrics_quality_df['estimator'].apply(lambda x: x[:-2])

    # создание сводной таблицы для анализа данных
    metrics_quality_df = pd.pivot_table(data = metrics_quality_df, index=['scoring', 'estimator', 'cv'], values=['train_metric_value', 'test_metric_value']).unstack(['cv'])
    metrics_quality_df = metrics_quality_df[['train_metric_value', 'test_metric_value']]

    return metrics_quality_df

In [ ]:
# Lasso
make_grid_search_cv(X, y, estimator=lasso, random_state=42, train_size=0.7, param_grid = {"alpha": np.logspace(-4, 3, 8), "positive": [True, False],
                                                                                          "fit_intercept": [True, False], "tol": np.logspace(-4, 3, 8),
                                                                                          "selection": ['cyclic', 'random']},
                    cvs_list = [3, 5, 7, 9], verbose=False, scorings_list = ['neg_mean_absolute_error', 'neg_mean_squared_error', 'r2'])

--------------------------------------------------------------------------------------------------------------
CV: 3, метрика: neg_mean_absolute_error
Лучшие параметры модели при переборе на обучающей выборке: {'alpha': 0.1, 'fit_intercept': True, 'positive': False, 'selection': 'random', 'tol': 0.01}
MAE на обучающей части: 3.336513459157023
MAE на тестовой части: 3.1938036858081715
--------------------------------------------------------------------------------------------------------------
CV: 3, метрика: neg_mean_squared_error
Лучшие параметры модели при переборе на обучающей выборке: {'alpha': 0.01, 'fit_intercept': True, 'positive': False, 'selection': 'random', 'tol': 0.0001}
MSE на обучающей части: 22.552767247510523
MSE на тестовой части: 21.567599789926295
--------------------------------------------------------------------------------------------------------------
CV: 3, метрика: r2
Лучшие параметры модели при переборе на обучающей выборке: {'alpha': 0.01, 'fit_intercept': T

train_metric_value                        \
cv                                                 3          5          7   
scoring                 estimator                                            
neg_mean_absolute_error Lasso               3.336513   3.335239   3.335239   
neg_mean_squared_error  Lasso              22.552767  22.553031  22.554271   
r2                      Lasso               0.742030   0.743499   0.743400   

                                             test_metric_value             \
cv                                         9                 3          5   
scoring                 estimator                                           
neg_mean_absolute_error Lasso       3.335239          3.193804   3.227374   
neg_mean_squared_error  Lasso      22.626298         21.567600  21.570194   
r2                      Lasso       0.741659          0.702181   0.711164   

                                                         
cv                                         7          9  
scoring                 estimator                        
neg_mean_absolute_error Lasso       3.227374   3.227374  
neg_mean_squared_error  Lasso      21.559140  21.960850  
r2                      Lasso       0.710666   0.700471

In [ ]:
# Linear Regression
make_grid_search_cv(X, y, estimator=lr, random_state=42, train_size=0.7, param_grid = {"fit_intercept": [True, False], "positive": [True, False], "copy_X": [True, False]},
                    cvs_list = [3, 5, 7, 10], verbose=False, scorings_list = ['neg_mean_absolute_error', 'neg_mean_squared_error', 'r2'])

--------------------------------------------------------------------------------------------------------------
CV: 3, метрика: neg_mean_absolute_error
Лучшие параметры модели при переборе на обучающей выборке: {'copy_X': True, 'fit_intercept': True, 'positive': False}
MAE на обучающей части: 3.3568267821682056
MAE на тестовой части: 3.162709871457406
--------------------------------------------------------------------------------------------------------------
CV: 3, метрика: neg_mean_squared_error
Лучшие параметры модели при переборе на обучающей выборке: {'copy_X': True, 'fit_intercept': True, 'positive': False}
MSE на обучающей части: 22.545481487421426
MSE на тестовой части: 21.51744423117722
--------------------------------------------------------------------------------------------------------------
CV: 3, метрика: r2
Лучшие параметры модели при переборе на обучающей выборке: {'copy_X': True, 'fit_intercept': True, 'positive': False}
R2 score: 0.7434997532004697
R2 score: 0.711226

train_metric_value             \
cv                                                       3          5    
scoring                 estimator                                        
neg_mean_absolute_error LinearRegression           3.356827   3.356827   
neg_mean_squared_error  LinearRegression          22.545481  22.545481   
r2                      LinearRegression           0.743500   0.743500   

                                                                \
cv                                               7          10   
scoring                 estimator                                
neg_mean_absolute_error LinearRegression   3.356827   3.356827   
neg_mean_squared_error  LinearRegression  22.545481  22.545481   
r2                      LinearRegression   0.743500   0.743500   

                                         test_metric_value             \
cv                                                      3          5    
scoring                 estimator                                       
neg_mean_absolute_error LinearRegression          3.162710   3.162710   
neg_mean_squared_error  LinearRegression         21.517444  21.517444   
r2                      LinearRegression          0.711226   0.711226   

                                                                
cv                                               7          10  
scoring                 estimator                               
neg_mean_absolute_error LinearRegression   3.162710   3.162710  
neg_mean_squared_error  LinearRegression  21.517444  21.517444  
r2                      LinearRegression   0.711226   0.711226

In [ ]:
# Ridge
make_grid_search_cv(X, y, estimator=ridge, random_state=42, train_size=0.7, param_grid = {"fit_intercept": [True, False], "copy_X": [True, False],
                                                                                          "alpha": np.logspace(-4, 5, 10), "tol": np.logspace(-4, 5, 10),
                                                                                          'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']},
                    cvs_list = [3, 5, 7, 10], verbose=False, scorings_list = ['neg_mean_absolute_error', 'neg_mean_squared_error', 'r2'])

--------------------------------------------------------------------------------------------------------------
CV: 3, метрика: neg_mean_absolute_error
Лучшие параметры модели при переборе на обучающей выборке: {'alpha': 1.0, 'copy_X': False, 'fit_intercept': True, 'solver': 'saga', 'tol': 0.1}
MAE на обучающей части: 3.308216361260028
MAE на тестовой части: 3.181529982839048
--------------------------------------------------------------------------------------------------------------
CV: 3, метрика: neg_mean_squared_error
Лучшие параметры модели при переборе на обучающей выборке: {'alpha': 1.0, 'copy_X': True, 'fit_intercept': True, 'solver': 'lsqr', 'tol': 0.001}
MSE на обучающей части: 22.547172600315005
MSE на тестовой части: 21.533527464241185
--------------------------------------------------------------------------------------------------------------
CV: 3, метрика: r2
Лучшие параметры модели при переборе на обучающей выборке: {'alpha': 0.01, 'copy_X': True, 'fit_intercept': True

train_metric_value                        \
cv                                                3          5          7    
scoring                 estimator                                            
neg_mean_absolute_error Ridge               3.308216   3.379804   3.513877   
neg_mean_squared_error  Ridge              22.547173  23.922321  22.645742   
r2                      Ridge               0.740774   0.738600   0.741095   

                                             test_metric_value             \
cv                                        10                3          5    
scoring                 estimator                                           
neg_mean_absolute_error Ridge       3.398434          3.181530   3.104690   
neg_mean_squared_error  Ridge      24.272475         21.533527  24.008532   
r2                      Ridge       0.715384          0.707475   0.694295   

                                                         
cv                                        7          10  
scoring                 estimator                        
neg_mean_absolute_error Ridge       3.188624   3.198109  
neg_mean_squared_error  Ridge      21.822903  23.806971  
r2                      Ridge       0.699104   0.632235

In [ ]:
# SGDRegressor
make_grid_search_cv(X, y, estimator=sgdreg, random_state=42, train_size=0.7, param_grid = {"loss": ['squared_error', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'],
                                                                                           "alpha": np.logspace(-4, 5, 10), "tol": np.logspace(-4, 5, 10),
                                                                                           "penalty": ['l2', 'l1', 'elasticnet', None]},
                    cvs_list = [3, 5, 7, 9], verbose=False, scorings_list = ['neg_mean_absolute_error', 'neg_mean_squared_error', 'r2'])

--------------------------------------------------------------------------------------------------------------
CV: 3, метрика: neg_mean_absolute_error
Лучшие параметры модели при переборе на обучающей выборке: {'alpha': 10.0, 'loss': 'epsilon_insensitive', 'penalty': None, 'tol': 0.0001}
MAE на обучающей части: 3.1371216355079614
MAE на тестовой части: 3.1692132701212348
--------------------------------------------------------------------------------------------------------------
CV: 3, метрика: neg_mean_squared_error
Лучшие параметры модели при переборе на обучающей выборке: {'alpha': 100.0, 'loss': 'squared_epsilon_insensitive', 'penalty': None, 'tol': 10.0}
MSE на обучающей части: 22.840672342088666
MSE на тестовой части: 23.0992743629252
--------------------------------------------------------------------------------------------------------------
CV: 3, метрика: r2
Лучшие параметры модели при переборе на обучающей выборке: {'alpha': 0.001, 'loss': 'squared_epsilon_insensitive', 'pe

train_metric_value                        \
cv                                                    3          5          7   
scoring                 estimator                                               
neg_mean_absolute_error SGDRegressor           3.137122   3.134880   3.146812   
neg_mean_squared_error  SGDRegressor          22.840672  23.340976  22.820329   
r2                      SGDRegressor           0.743089   0.739692   0.739580   

                                                test_metric_value             \
cv                                            9                 3          5   
scoring                 estimator                                              
neg_mean_absolute_error SGDRegressor   3.146605          3.169213   3.172484   
neg_mean_squared_error  SGDRegressor  22.814725         23.099274  23.407989   
r2                      SGDRegressor   0.736574          0.707433   0.707663   

                                                            
cv                                            7          9  
scoring                 estimator                           
neg_mean_absolute_error SGDRegressor   3.184233   3.183024  
neg_mean_squared_error  SGDRegressor  22.206813  22.042852  
r2                      SGDRegressor   0.690158   0.693419

In [ ]:
# SVR
make_grid_search_cv(X, y, estimator=svr, random_state=42, train_size=0.7, param_grid = {"degree": [3, 5, 7, 9], "gamma": ['scale', 'auto'],
                                                                                        "C": np.logspace(1, 5, 30), "tol": np.logspace(1, 5, 30)},
                    cvs_list = [3, 5, 7, 9], verbose=False, scorings_list = ['neg_mean_absolute_error', 'neg_mean_squared_error', 'r2'])

--------------------------------------------------------------------------------------------------------------
CV: 3, метрика: neg_mean_absolute_error
Лучшие параметры модели при переборе на обучающей выборке: {'C': 92.36708571873861, 'degree': 3, 'gamma': 'scale', 'tol': 10.0}
MAE на обучающей части: 1.8020249741210568
MAE на тестовой части: 2.504719831686022
--------------------------------------------------------------------------------------------------------------
CV: 3, метрика: neg_mean_squared_error
Лучшие параметры модели при переборе на обучающей выборке: {'C': 126.89610031679221, 'degree': 3, 'gamma': 'auto', 'tol': 10.0}
MSE на обучающей части: 5.082500761493653
MSE на тестовой части: 14.80580853238555
--------------------------------------------------------------------------------------------------------------
CV: 3, метрика: r2
Лучшие параметры модели при переборе на обучающей выборке: {'C': 126.89610031679221, 'degree': 3, 'gamma': 'auto', 'tol': 10.0}
R2 score: 0.942176

train_metric_value                      \
cv                                                 3         5         7   
scoring                 estimator                                          
neg_mean_absolute_error SVR                 1.802025  1.556973  1.791497   
neg_mean_squared_error  SVR                 5.082501  5.146585  4.178190   
r2                      SVR                 0.942176  0.941447  0.952465   

                                            test_metric_value             \
cv                                        9                 3          5   
scoring                 estimator                                          
neg_mean_absolute_error SVR        1.875474          2.504720   2.394901   
neg_mean_squared_error  SVR        5.082501         14.805809  13.935307   
r2                      SVR        0.942176          0.801299   0.812982   

                                                         
cv                                         7          9  
scoring                 estimator                        
neg_mean_absolute_error SVR         2.672068   2.681858  
neg_mean_squared_error  SVR        13.441336  14.805809  
r2                      SVR         0.819611   0.801299